In [2]:
import pandas as pd
file = "Training/Features_Variant_1.csv"
df = pd.read_csv(file, header=None)
df.sample(n=5)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
37199,78923,0,25345,24,7.0,135.0,69.571429,66.5,33.749528,1.0,...,0,1,0,1,0,0,0,0,0,1
5225,6141231,108622,290050,36,0.0,1514.0,242.430168,165.0,243.201641,0.0,...,0,0,0,0,0,0,0,0,1,0
11590,368445,0,30549,9,0.0,785.0,76.225000,31.5,146.404660,0.0,...,0,0,0,0,0,1,0,0,0,9
37301,1548,0,581,16,0.0,173.0,8.071429,3.0,20.322979,0.0,...,0,0,0,0,0,0,0,1,0,0
18854,55275,0,1834,13,0.0,411.0,6.313514,2.0,30.640356,0.0,...,1,0,1,0,0,0,0,0,0,0


In [3]:
#dataset shape
print("Dataset has {} entries and {} features".format(*df.shape))

Dataset has 40949 entries and 54 features


In [4]:
#feature extraction and splitting data
X, y = df.loc[:,:52].values, df.loc[:,53].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1, random_state=42)

In [9]:
#fixed xgboost import issue
!pip install xgboost


distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.


In [10]:
#load data in dmatrices
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [11]:
#use MAE as main metric
from sklearn.metrics import mean_absolute_error
import numpy as np
# "Learn" the mean from the training data
mean_train = np.mean(y_train)

# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train

# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)

print("Baseline MAE is {:.2f}".format(mae_baseline))

Baseline MAE is 11.31


In [ ]:
#tuning xgboost
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}
params['eval_metric'] = "mae"
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)


cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

cv_results

[0]	Test-mae:5.97478
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:5.03359
[2]	Test-mae:4.64572
[3]	Test-mae:4.42331
[4]	Test-mae:4.39328
[5]	Test-mae:4.35544
[6]	Test-mae:4.31315
[7]	Test-mae:4.33087
[8]	Test-mae:4.37164
[9]	Test-mae:4.38774
[10]	Test-mae:4.39443
[11]	Test-mae:4.40661
[12]	Test-mae:4.39124
[13]	Test-mae:4.39088
[14]	Test-mae:4.39827
[15]	Test-mae:4.39104
[16]	Test-mae:4.40307
Stopping. Best iteration:
[6]	Test-mae:4.31315



In [ ]:
print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))